In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

从2024年9月15日开始，您只能通过转移到Vertex AI Gemini提示和调整来定制分类、实体抽取和情感分析模型。针对Vertex AI AutoML的文本分类、实体抽取和情感分析目标进行训练或更新的模型将不再提供。您可以继续使用现有的Vertex AI AutoML文本目标直到2025年6月15日。有关Gemini如何通过改进的提示功能提供增强用户体验的更多信息，请参阅[调整简介](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-gemini-overview)。

# Vertex AI SDK：为在线预测训练AutoML文本情感分析模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_text_sentiment_analysis_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_text_sentiment_analysis_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_text_sentiment_analysis_online.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI SDK来训练和部署一个[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)文本情感分析模型，并从中获得在线预测。

了解更多关于[文本数据情感分析](https://cloud.google.com/vertex-ai/docs/training-overview#sentiment_analysis_for_text)。

### 目标

在本教程中，您将学习如何使用 Vertex AI SDK 从 Python 脚本创建一个 AutoML 文本情感分析模型，并部署它以进行在线预测。您还可以使用 `gcloud` 命令行工具或在 Cloud 控制台上在线创建和部署模型。

本教程使用以下 Google Cloud ML 服务和资源：
- Vertex AI 数据集
- Vertex AI 训练 (AutoML)
- Vertex AI 模型注册表
- Vertex AI 终端

执行的步骤包括：

- 创建一个 `Vertex AI 数据集` 资源。
- 为数据集创建一个 AutoML 模型的训练作业。
- 查看模型评估指标。
- 部署 `Vertex AI 模型` 资源到一个服务化的 `Vertex AI 终端`。
- 发送一个预测请求到部署的模型。
- 从终端将模型下线。
- 执行清理过程。

数据集

本教程使用的数据集是[Crowdflower Claritin-Twitter数据集](https://data.world/crowdflower/claritin-twitter)，其中包含带有情感标签的推文，作者的性别，以及他们是否提到FDA报告的十大不良事件中的任何一个。您在本教程中使用的数据集版本存储在公共云存储桶中。在本教程中，您将使用推文数据在Google Cloud平台上构建一个AutoML文本情感分析模型。

### 成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装最新版本的用于 Python 的 Vertex AI SDK。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage

只有 Colab：取消注释下面的单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的 Google Cloud 帐户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照下面相关的说明进行操作。

**1. Vertex AI 工作台**
* 无需操作，您已经通过身份验证。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务帐户授予云存储权限。

### 创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 定义常数

设置在本教程中使用的常数。

In [ ]:
# Set the location of the CSV index file in Cloud Storage.
IMPORT_FILE = "gs://cloud-samples-data/language/claritin.csv"
# Set the max. sentiment score
SENTIMENT_MAX = 4

## 快速查看您的数据

本教程使用存储在公共云存储桶中的`Crowdflower Claritin-Twitter`数据集的一个版本，使用CSV索引文件。

首先快速查看数据。此外，通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后打印前几行。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

## 创建数据集

现在，使用`TextDataset`类的`create`方法创建一个`Vertex AI Dataset`资源，该方法需要以下参数：

- `display_name`：数据集资源的可读名称。
- `gcs_source`：要将数据项目导入数据集资源的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项目的数据标记模式。

这个操作可能需要几分钟。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="Crowdflower Claritin-Twitter",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.sentiment,
)

print(dataset.resource_name)

## 创建和运行训练任务

在这一部分，为了训练一个AutoML模型，您需要执行以下步骤：

1）创建一个训练任务。
2）运行该任务。

### 创建训练任务

使用`AutoMLTextTrainingJob`类创建一个AutoML训练任务，包括以下参数：

- `display_name`：训练任务资源的可读名称。
- `prediction_type`：要为模型训练的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是分类任务，是否单标签（False）或多标签（True）。
- `sentiment_max`：如果是情感分析任务，则为情感值的最大值。

In [ ]:
job = aiplatform.AutoMLTextTrainingJob(
    display_name="claritin",
    prediction_type="sentiment",
    sentiment_max=SENTIMENT_MAX,
)

print(job)

### 运行训练作业

接下来，通过调用`run`方法来运行训练作业，并传入以下参数：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 已训练模型的人类可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（保留数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。

完成`run`方法后将返回`Model`资源。

训练流水线的执行需要最多180分钟。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name="claritin",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 检查模型评估分数

当您的模型训练完成后，您可以查看评估分数。

首先，您需要获取对新创建的模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型并进行过滤。

In [ ]:
# Get model resource ID
models = aiplatform.Model.list(filter="display_name=claritin")

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

部署模型

接下来，部署您的模型以提供在线预测。要部署模型，您需要调用模型资源的`deploy`方法，该方法将返回部署的终端节点。

**注意：**通常，在模型部署之前会创建一个终端节点，并且在模型部署时将其作为参考。默认情况下，当没有给定终端节点引用时，`deploy()`方法会创建一个终端节点。

In [ ]:
endpoint = model.deploy()

发送在线预测请求

在这一步中，您从数据集中准备一些测试实例，并向部署的模型发送在线预测请求。

### 创建测试实例

您可以从数据集中随意选择一个示例作为测试项。不用担心这个示例很可能在训练模型时被使用过。这只是为了演示如何进行预测。

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
if len(test_item[0]) == 3:
    _, test_item, test_label, max = str(test_item[0]).split(",")
else:
    test_item, test_label, max = str(test_item[0]).split(",")

print(test_item, test_label)

### 发出预测请求

现在您的模型已部署到一个端点，您可以向端点资源发送在线预测请求。

#### 请求格式

每个实例的格式应该如下的JSON格式：

     { 'content': text_string }

由于`predict()`方法可以接受多个实例，因此请将您的请求发送为一个测试实例的列表。

#### 响应

从`predict()`调用的响应是一个Python字典，其中包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `sentiment`：情感值。
- `deployed_model_id`：执行预测的Vertex AI部署的`Model`资源的标识符。

In [ ]:
instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(prediction)

将模型取消部署

在您探索预测结果之后，您可以从`Endpoint`资源中取消部署模型。这将取消所有计算资源，并结束已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理本项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的单个资源：

- Vertex AI数据集
- Vertex AI模型
- Vertex AI终端
- AutoML训练作业
- 云存储桶（将 `delete_bucket` 设置为 **True** 来删除桶）

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the Cloud storage bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI